In [1]:
import sys
sys.path.append('../src')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load

In [2]:
T = 30
bus_network = gk.generate_kvadraturen_w_bus(T)

In [3]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 27.550016403198242, should wait for 29.835927963256836 seconds


In [4]:
# 27.550016403198242, should wait for 29.835927963256836 seconds

In [5]:
print(bus_delays)

{0: [tensor(0.), tensor(0.), tensor(0.), tensor(0.)], 1: [tensor(23.8359, grad_fn=<AddBackward0>), tensor(0.), tensor(0.), tensor(0.)], 2: [tensor(0.), tensor(0.), tensor(0.), tensor(0.)], 3: [tensor(0.), tensor(0.), tensor(0.), tensor(0.)]}


In [6]:
objective_fnc = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        objective_fnc = objective_fnc + bus_delays[i][j]
print(objective_fnc)

tensor(23.8359, grad_fn=<AddBackward0>)


In [7]:
objective_fnc.backward()

In [8]:
adgHSKKL

NameError: name 'adgHSKKL' is not defined

In [ ]:
n = 4
m = 3
A = torch.ones((n, m))
B = torch.tensor([[1.3, 5.2, 12.4],
                  [24.1, 0.04, 0.5],
                  [12.1, 12.4, 22.2],
                  [1.2, 4.6, 7.8]])
i = 3
j = 1
tot_sum = torch.tensor(0.0)
mask = torch.ones(n, dtype=torch.bool)
mask[i] = False
filtered_tensor = torch.cat((B[:i,j], B[i+1:,j]))
for l in range(n):
    if l != i:
        tot_sum += B[l,j]
second_sum = torch.sum(B[mask, j])

tot_sum
second_sum - tot_sum

tensor(0.)

In [ ]:
densities_loaded = load.convert_from_tensor(densities)
queues_loaded = load.convert_from_tensor(queues)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)

In [ ]:
bus_delays_dict = {0: [float(l.detach()) for l in bus_delays[0]],
                   1: [float(l.detach()) for l in bus_delays[1]],
                   2: [float(l.detach()) for l in bus_delays[2]],
                   3: [float(l.detach()) for l in bus_delays[3]]}
                   

In [ ]:
# Write above dictionaries to json file
import json
# with open("kvadraturen_results_minimal_1000_variable.json", 'w') as fd:
#     fd.write(json.dumps([densities, queues, bus_lengths, bus_delays_dict]))
with open("kvadraturen_roundabout_750.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, queues_loaded, bus_lengths_loaded, bus_delays_dict]))

In [ ]:
import json
print("Loading results...")
f = open("kvadraturen_roundabout_750.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [ ]:
import openGLUtilsGif as opgl
opgl.draw_busses_w_densities(bus_network, [bus_fw, bus_bw, bus_fw_2, bus_bw_2], bus_lengths,
                                    densities, output_name="test_roundabout.gif")

Maximum and minimum x points: 7, -0.5
Maximum and minimum y points: -0.4, 8.9
Window created


: 